# (Multi-observable) ODMD Utilities

In [1]:
from __future__ import division
import sys,math,random,numpy as np,numpy.typing as npt,scipy,itertools,tqdm,warnings,matplotlib.pylab as pl,functools as ft
from pylab import meshgrid,cm,imshow,contour,clabel,colorbar,axis,title,show
from matplotlib import pyplot as plt, cm as cm, mlab as ml, rc
from scipy.linalg import svd,eig,eigh,inv,norm,toeplitz,circulant,lstsq
from scipy.io import savemat, loadmat

###### Scaling factor for Hamiltonian with spectrum in [-a, a] ######
a = 3*np.pi/4

def scale(E):
    """
    Scale the Hamiltonian spectrum.

    Parameters:
    - E: Energy spectrum of Hamiltonian (E_0, E_1, ..., E_{Hilbert space dimension})
    
    Returns:
    - Scaled energy spectrum in [-a, a] for a < π
      Rescaling factor for proper conversion of final energy estimate
    """
    
    E_center = E - (E.min()+E.max())/2
    return a*E_center/np.abs(E_center).max(), np.abs(E_center).max()/a


###### Generation of reference state with prescribed target eigenstate overlap for ODMD ######
def state_ref(target_overlap, E):
    """
    Generate a reference state with prescribed eigenstate overlap.

    Parameters:
    - target_overlap: Prescribed overlaps (p_0, p_1, ..., p_{# target eigenstates})
    - E: Energy spectrum of Hamiltonian
    
    Returns:
    - state: reference state with specified overlaps on target eigenstates 
             and uniform overlap over remaining eigenstates
    """
    state = np.zeros(len(E))
    state[:len(target_overlap)] = np.sqrt(target_overlap)
    state[len(target_overlap):] = np.sqrt((1- np.sum(target_overlap))/(len(E)-len(target_overlap))) 
    return state


###### Generation of overlap data (observables) for ODMD ######
def S_gen(ref_state, time_grid, E, eps=0):
    """
    Generate an overlap trajectory <ɸ|exp(-iHt)|ɸ>.

    Parameters:
    - time_grid: General time grid (t_0 = 0, t_1, ..., t_{NT_max-1})
    - E: Energy spectrum of Hamiltonian
    - eps: Standard deviation of gaussian noises on matrix elements
    
    Returns:
    - S: Overlap trajectory [s(t0), s(t1), ...]
    """
    T = len(time_grid)
    S = np.zeros((2, T))
    for t in range(T):
        phases_t = np.exp(-1j * E * time_grid[t])
        overlap = np.sum(phases_t * ref_state**2)
        S[0,t], S[1,t] = overlap.real, overlap.imag
    S[:,1:] = S[:,1:] + np.random.normal(0, eps, (2, T-1)) 
    return S

In [2]:
###### (Multi-observable) ODMD for eigenenergy estimation ######
def BHankel(data):
    """
    Construct a block Hankel matrix from input data.
    
    Parameters:
    - data: Input data matrix (rows are variables, columns are time snapshots)
    - m: Block size for the Hankel matrix
    
    Returns:
    - H: Block Hankel matrix
    """
    rows, cols = data.shape
    m = cols//3

    # Initialize the Hankel matrix
    X = np.zeros((rows * m, cols + 1 - m), dtype=complex)

    # Loop through each column and fill the matrix
    for i in range(cols + 1 - m):
        temp = data[:, i:i + m].T
        X[:, i] = temp.ravel()  # Flatten temp and store in X
    return X

def MODMD(data, dt, tol=1E-8, eigid=0):
    """
    Run (multi-observable) ODMD with SVD truncation.
    
    Parameters:
    - data: Input data matrix (each column is a snapshot in time)
            We have standard ODMD when we have a single row of time snapshots
    - dt: Time between data snapshots
    - tol: Tolerance for rank truncation
    - eigid: Number of eigenvalues to return
    
    Returns:
    - E_approx: Approximate eigenenergies as the sorted imaginary part of ODMD eigenvalues
    """
    
    # Shifted data matrices (block Hankel)
    X = BHankel(data)
    X1 = X[:, :-1]
    X2 = X[:, 1:]
    
    # SVD of X1
    U, S, Vh = svd(X1, full_matrices=False)
    
    # Rank truncation
    r = np.sum(S > tol * S[0])
    U = U[:, :r]
    S = S[:r]
    V = Vh[:r, :].conj().T
    S_inv = np.diag(1/S) 
    
    # DMD computation
    Atilde = np.dot(np.dot(U.T, X2), np.dot(V, S_inv))
    
    # Eigenvalue computation
    mu = eig(Atilde)[0]
    omega = np.log(mu) / dt
    Etilde = np.sort(-np.imag(omega))
    
    # Return the approximate eigenvalues
    return Etilde[eigid]